<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/master/Training_Clustering_Test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
 # Run this and follow the instructions:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Once mounted:
# You may need to change the paths to the correct one on your drive
!cp /content/drive/My\ Drive/Project3 -r /content/

In [16]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (124 kB/s)
Reading package lists... Done


In [17]:
# Start a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [27]:
# Note this is an RDD approach, not dataframe. API is slightly different for dataframe
from pyspark.mllib.feature import Word2Vec
inp = spark.sparkContext.textFile("Project3/yelp-100k.txt").map(lambda row: row.split(" "))
word2vec = Word2Vec()

In [28]:
# Train the model
model = word2vec.fit(inp)

In [29]:
synonyms = model.findSynonyms('restaurant', 5)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))

location: 0.7936934232711792
store: 0.7661047577857971
gym: 0.7582272291183472
hotel: 0.7312384843826294
establishment: 0.7134353518486023


In [30]:
def cosine_similarity(x, y):
    return x.dot(y)/(x.norm(2)*y.norm(2))

In [34]:
x = model.transform('location')
y = model.transform('restaurant')
cosine_similarity(x, y)

0.7936935055394885

In [35]:
synonyms = model.findSynonyms('diner', 2)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))

shitty: 0.7646585702896118
slice,: 0.7196394205093384


In [38]:
def cosine_similarity(x, y):
    return x.dot(y)/(x.norm(2)*y.norm(2))

x = model.transform('diner')
y = model.transform('slice,')
cosine_similarity(x, y)

0.719639502741139